In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nfl_data_py as nfl

In [3]:
base_pbp = nfl.import_pbp_data([2023], downcast=True, cache=False, alt_path=None)

2023 done.
Downcasting floats.


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
pd.set_option('display.max_seq_items', None)

In [6]:
base_pbp.columns

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp',
       'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet',
       'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle',
       'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length',
       'pass_location', 'air_yards', 'yards_after_catch', 'run_location',
       'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result',
       'two_point_conv_result', 'home_timeouts_remaining',
       'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team',
       'td_player_name', 'td_player_id', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'total_home_score', 'total_away_score',
       'posteam_score'

In [7]:
# Step 2: Extract unique teams
teams = set(base_pbp['home_team'].unique()).union(set(base_pbp['away_team'].unique()))

# Step 3: Assign unique team_id to each team
team_id_mapping = {team: i + 1 for i, team in enumerate(sorted(teams))}  # Assigning sequential IDs starting from 1

# Step 4: Add home_team_id and away_team_id to the pbp data
base_pbp['home_team_id'] = base_pbp['home_team'].map(team_id_mapping)
base_pbp['away_team_id'] = base_pbp['away_team'].map(team_id_mapping)

# Display the updated pbp data
print(base_pbp)

       play_id          game_id old_game_id home_team away_team season_type  \
0          1.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
1         39.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
2         55.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
3         77.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
4        102.0  2023_01_ARI_WAS  2023091007       WAS       ARI         REG   
...        ...              ...         ...       ...       ...         ...   
49660   4791.0    2023_22_SF_KC  2024021100        KC        SF        POST   
49661   4813.0    2023_22_SF_KC  2024021100        KC        SF        POST   
49662   4835.0    2023_22_SF_KC  2024021100        KC        SF        POST   
49663   4860.0    2023_22_SF_KC  2024021100        KC        SF        POST   
49664   4881.0    2023_22_SF_KC  2024021100        KC        SF        POST   

       week posteam posteam_type defteam side_of_fi

C:\Users\bhugg\AppData\Local\Temp\ipykernel_25512\1391511132.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_pbp['home_team_id'] = base_pbp['home_team'].map(team_id_mapping)
C:\Users\bhugg\AppData\Local\Temp\ipykernel_25512\1391511132.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_pbp['away_team_id'] = base_pbp['away_team'].map(team_id_mapping)


In [8]:
# Step 1: Calculate average home_team_epa and average away_team_epa
average_home_team_epa = base_pbp.groupby('home_team')['epa'].mean()
average_away_team_epa = base_pbp.groupby('away_team')['epa'].mean()

# Step 2: Merge average home_team_epa and average away_team_epa with base_pbp
base_pbp = base_pbp.merge(average_home_team_epa.rename('average_home_team_epa'), left_on='home_team', right_index=True)
base_pbp = base_pbp.merge(average_away_team_epa.rename('average_away_team_epa'), left_on='away_team', right_index=True)

In [9]:
# firsto = base_pbp[base_pbp['game_id'] == '2023_01_ARI_WAS']

In [10]:
twothree = base_pbp[['week', 'season_type', 'posteam', 'average_home_team_epa', 'average_away_team_epa', 'home_team_id', 'home_team', 'away_team_id', 'away_team', 'game_id', 'game_seconds_remaining', 'total', 'spread_line','total_line','home_wp', 'away_wp', 'safety', 'fumble', 'interception', 'qb_hit', 'total_home_score', 'total_away_score', 'score_differential']]
twothree.value_counts()

#use first for all these columns in aggregation

week  season_type  posteam  average_home_team_epa  average_away_team_epa  home_team_id  home_team  away_team_id  away_team  game_id          game_seconds_remaining  total  spread_line  total_line  home_wp   away_wp   safety  fumble  interception  qb_hit  total_home_score  total_away_score  score_differential
17    REG          DET       0.020359               0.028253              9             DAL        11            DET        2023_17_DET_DAL  23.0                    39      5.5         53.0        0.663312  0.336688  0.0     0.0     0.0           0.0     20.0              19.0              -1.0                  3
7     REG          NE       -0.110698              -0.001322              22            NE         4             BUF        2023_07_BUF_NE   456.0                   54     -7.5         41.0        0.956606  0.043394  0.0     0.0     0.0           0.0     22.0              10.0               12.0                 3
4     REG          MIN      -0.051170              -0.061643

In [11]:
twothree.sort_values(by = 'game_seconds_remaining', ascending = False)

,week,season_type,posteam,average_home_team_epa,average_away_team_epa,home_team_id,home_team,away_team_id,away_team,game_id,game_seconds_remaining,total,spread_line,total_line,home_wp,away_wp,safety,fumble,interception,qb_hit,total_home_score,total_away_score,score_differential
0,1,REG,None,-0.012497,-0.026100,32,WAS,1,ARI,2023_01_ARI_WAS,3600.0,36,7.0,38.0,0.546262,0.453738,NaN,NaN,NaN,NaN,0.0,0.0,NaN
33129,13,REG,None,-0.110698,-0.037910,22,NE,18,LAC,2023_13_LAC_NE,3600.0,6,-4.5,38.5,0.546262,0.453738,NaN,NaN,NaN,NaN,0.0,0.0,NaN
27007,11,REG,None,0.001483,-0.037910,12,GB,18,LAC,2023_11_LAC_GB,3600.0,43,-3.0,44.0,0.546262,0.453738,NaN,NaN,NaN,NaN,0.0,0.0,NaN
3882,2,REG,LAC,0.003156,-0.037910,31,TEN,18,LAC,2023_02_LAC_TEN,3600.0,51,-2.5,45.5,0.566792,0.433208,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,2,REG,LAC,0.003156,-0.037910,31,TEN,18,LAC,2023_02_LAC_TEN,3600.0,51,-2.5,45.5,0.566792,0.433208,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31578,13,REG,PIT,-0.062948,-0.026100,27,PIT,1,ARI,2023_13_ARI_PIT,NaN,34,6.0,41.0,0.731358,0.268642,NaN,NaN,NaN,NaN,3.0,10.0,-7.0
31579,13,REG,PIT,-0.062948,-0.026100,27,PIT,1,ARI,2023_13_ARI_PIT,NaN,34,6.0,41.0,0.731358,0.268642,NaN,NaN,NaN,NaN,3.0,10.0,-7.0
31603,13,REG,ARI,-0.062948,-0.026100,27,PIT,1,ARI,2023_13_ARI_PIT,NaN,34,6.0,41.0,0.851048,0.148952,NaN,NaN,NaN,NaN,3.0,17.0,14.0
22697,9,REG,MIA,-0.030520,0.041990,16,KC,20,MIA,2023_09_MIA_KC,NaN,35,1.0,51.0,0.955770,0.044230,NaN,NaN,NaN,NaN,21.0,0.0,-21.0


In [12]:
# Group by 'week' and 'game_id'
grouped = twothree.groupby(['week', 'game_id', 'season_type'])

# Aggregating data for each group
agg_df = grouped.agg({
    'home_team_id': 'first',
    'home_team': 'first',
    'away_team_id': 'first',
    'away_team': 'first',
    'total': 'first',
    'spread_line': 'first',
    'total_line': 'first',
    'average_home_team_epa': 'mean',
    'average_away_team_epa': 'mean',
    'home_wp': 'first',
    'away_wp': 'first',
    'fumble': [lambda x: x[twothree['posteam'] == twothree['away_team']].sum(), lambda x: x[twothree['posteam'] == twothree['home_team']].sum()],
    'interception': [lambda x: x[twothree['posteam'] == twothree['away_team']].sum(), lambda x: x[twothree['posteam'] == twothree['home_team']].sum()],
    'safety': [lambda x: x[twothree['posteam'] == twothree['away_team']].sum(), lambda x: x[twothree['posteam'] == twothree['home_team']].sum()],
    'qb_hit': [lambda x: x[twothree['posteam'] == twothree['away_team']].sum(), lambda x: x[twothree['posteam'] == twothree['home_team']].sum()],
    'total_home_score': 'last',
    'total_away_score': 'last',
    'score_differential': 'last'
})

# Renaming the columns for clarity
agg_df.columns = [
    'home_team_id', 'home_team', 'away_team_id', 'away_team',
    'total', 'home_spread', 'total_line', 'average_home_team_epa',
    'average_away_team_epa', 'home_wp', 'away_wp', 
    'away_earned_fumbles', 'home_earned_fumbles', 'away_earned_interceptions', 
    'home_earned_interceptions', 'away_earned_safeties', 'home_earned_safeties', 
    'away_earned_qb_hits', 'home_earned_qb_hits', 'total_home_score',
    'total_away_score', 'score_differential'
]

# Resetting the index
agg_df.reset_index(inplace=True)

In [13]:
agg_df['away_spread'] = agg_df['home_spread'] * -1

In [14]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
agg_df

,week,game_id,season_type,home_team_id,home_team,away_team_id,away_team,total,home_spread,total_line,average_home_team_epa,average_away_team_epa,home_wp,away_wp,away_earned_fumbles,home_earned_fumbles,away_earned_interceptions,home_earned_interceptions,away_earned_safeties,home_earned_safeties,away_earned_qb_hits,home_earned_qb_hits,total_home_score,total_away_score,score_differential,away_spread
0,1,2023_01_ARI_WAS,REG,32,WAS,1,ARI,36,7.0,38.0,-0.012497,-0.026100,0.546262,0.453738,2.0,2.0,0.0,1.0,0.0,0.0,5.0,6.0,20.0,16.0,4.0,-7.0
1,1,2023_01_BUF_NYJ,REG,25,NYJ,4,BUF,38,-2.5,44.5,-0.119031,-0.001322,0.566792,0.433208,1.0,1.0,3.0,1.0,0.0,0.0,9.0,7.0,22.0,16.0,6.0,2.5
2,1,2023_01_CAR_ATL,REG,2,ATL,5,CAR,34,3.5,40.5,-0.042606,-0.063801,0.546262,0.453738,2.0,1.0,2.0,0.0,0.0,0.0,7.0,4.0,24.0,10.0,14.0,-3.5
3,1,2023_01_CIN_CLE,REG,8,CLE,7,CIN,27,-1.0,46.5,-0.163831,-0.008378,0.566792,0.433208,0.0,1.0,0.0,1.0,0.0,0.0,10.0,5.0,24.0,3.0,-21.0,1.0
4,1,2023_01_DAL_NYG,REG,24,NYG,9,DAL,40,-3.5,44.5,-0.153143,0.020445,0.546262,0.453738,1.0,5.0,0.0,2.0,0.0,0.0,3.0,12.0,0.0,40.0,-40.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,20,2023_20_KC_BUF,POST,4,BUF,16,KC,51,2.5,45.5,0.018987,-0.015578,0.546262,0.453738,2.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,24.0,27.0,3.0,-2.5
281,20,2023_20_TB_DET,POST,11,DET,30,TB,54,6.0,49.5,-0.005793,-0.004263,0.546262,0.453738,0.0,0.0,2.0,0.0,0.0,0.0,7.0,5.0,31.0,23.0,8.0,-6.0
282,21,2023_21_DET_SF,POST,29,SF,11,DET,65,7.5,52.5,0.061006,0.028253,0.566792,0.433208,1.0,0.0,0.0,1.0,0.0,0.0,5.0,5.0,34.0,31.0,3.0,-7.5
283,21,2023_21_KC_BAL,POST,3,BAL,16,KC,27,4.5,44.0,-0.008949,-0.015578,0.546262,0.453738,2.0,2.0,0.0,1.0,0.0,0.0,7.0,7.0,10.0,17.0,7.0,-4.5


In [16]:
for week in twothree['week'].unique():
    week_data = twothree[twothree['week'] == week]
    
    # Sort and rank for home teams
    home_ranked = week_data[['home_team', 'average_home_team_epa']].sort_values(by='average_home_team_epa', ascending=False)
    home_ranked['home_epa_rank'] = home_ranked['average_home_team_epa'].rank(method='average', ascending=False)

    # Sort and rank for away teams
    away_ranked = week_data[['away_team', 'average_away_team_epa']].sort_values(by='average_away_team_epa', ascending=False)
    away_ranked['away_epa_rank'] = away_ranked['average_away_team_epa'].rank(method='average', ascending=False)

    # Merge ranks back into the main DataFrame
    twothree = twothree.merge(home_ranked[['home_team', 'home_epa_rank']], on='home_team')
    twothree = twothree.merge(away_ranked[['away_team', 'away_epa_rank']], on='away_team')

MemoryError: Unable to allocate 17.5 GiB for an array with shape (11, 427335579) and data type float32